# به نام خدا

# فاز 1

In [58]:
import cv2
import os
import xml.etree.ElementTree as ET
from tqdm import tqdm

In [59]:


def parse_annotation(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    plates = []
    for obj in root.findall('object'):
        name = obj.find('name').text
        if name.lower() == 'vehicle plate':
            bndbox = obj.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)
            plates.append((xmin, ymin, xmax, ymax))
    
    return plates

def extract_plates(image_path, annotation_path, output_path):
    # خواندن تصویر
    image = cv2.imread(image_path)
    if image is None:
        print(f"خطا در خواندن تصویر: {image_path}")
        return
    
    # خواندن آنوتیشن‌ها
    plates = parse_annotation(annotation_path)
    if not plates:
        print(f"هیچ پلاکی در فایل آنوتیشن یافت نشد: {annotation_path}")
        return
    
    # استخراج و ذخیره هر پلاک
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    for i, (xmin, ymin, xmax, ymax) in enumerate(plates):
        plate_img = image[ymin:ymax, xmin:xmax]
        
        # ذخیره پلاک استخراج شده
        output_file = os.path.join(output_path, f"{base_name}.png")
        cv2.imwrite(output_file, plate_img)

def process_dataset(base_dir, output_dir):
    # مسیرهای ورودی
    images_dir = os.path.join(base_dir, "Vehicle Plates", "Vehicle Plates")
    annotations_dir = os.path.join(base_dir, "Vehicle Plates annotations", "Vehicle Plates annotations")
    
    # بررسی وجود پوشه‌ها
    if not os.path.exists(images_dir):
        print(f"پوشه تصاویر یافت نشد: {images_dir}")
        return
    if not os.path.exists(annotations_dir):
        print(f"پوشه آنوتیشن‌ها یافت نشد: {annotations_dir}")
        return
    
    # لیست تمام فایل‌های تصویر
    image_files = [f for f in os.listdir(images_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    # ایجاد پوشه خروجی اگر وجود نداشته باشد
    os.makedirs(output_dir, exist_ok=True)
    
    # پردازش هر تصویر با نوار پیشرفت
    for img_file in tqdm(image_files, desc="پردازش تصاویر"):
        base_name = os.path.splitext(img_file)[0]
        image_path = os.path.join(images_dir, img_file)
        annotation_path = os.path.join(annotations_dir, f"{base_name}.xml")
        
        if os.path.exists(annotation_path):
            extract_plates(image_path, annotation_path, output_dir)
        else:
            print(f"فایل آنوتیشن یافت نشد: {annotation_path}")

if __name__ == "__main__":
    # مسیرهای ورودی و خروجی
    base_dir = "Plates2"
    output_dir = "output_phase1"
    
    # اجرای پردازش
    process_dataset(base_dir, output_dir)
    
    print(f"پردازش فاز اول با موفقیت به پایان رسید. پلاک‌های استخراج شده در پوشه '{output_dir}' ذخیره شدند.")

پردازش تصاویر: 100%|██████████| 217/217 [00:00<00:00, 301.14it/s]

پردازش فاز اول با موفقیت به پایان رسید. پلاک‌های استخراج شده در پوشه 'output_phase1' ذخیره شدند.


# فاز 2

### درست کردن کجی و سیاه سفید کردن

In [18]:
import os
import cv2
import numpy as np

input_folder = 'output_phase1'
output_folder = 'output_phase2'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

def angle_from_min_area_rect(thresh):
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return None
    largest = max(contours, key=cv2.contourArea)
    rect = cv2.minAreaRect(largest)
    angle = rect[-1]
    if angle < -45:
        angle += 90
    elif angle > 45:
        angle -= 90
    return angle

def angle_from_hough(gray):
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=50, minLineLength=30, maxLineGap=10)
    if lines is None:
        return None
    angles = []
    for line in lines:
        x1, y1, x2, y2 = line[0]
        angle = np.degrees(np.arctan2(y2 - y1, x2 - x1))
        if -45 < angle < 45:
            angles.append(angle)
    if not angles:
        return None
    return np.median(angles)

def correct_skew_combined(gray_image, angle_threshold=3):
    # Preprocess
    _, thresh = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    thresh = 255 - thresh

    # Step 1: try minAreaRect
    angle = angle_from_min_area_rect(thresh)

    # Step 2: fallback to HoughLines if angle too small or None
    if angle is None or abs(angle) < angle_threshold:
        angle = angle_from_hough(gray_image)

    # اگر همچنان زاویه معنادار نبود، تصویر را برگردان
    if angle is None or abs(angle) < angle_threshold:
        return gray_image

    # Rotate
    (h, w) = gray_image.shape
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(gray_image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

# پردازش همه تصاویر
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
        path = os.path.join(input_folder, filename)
        image = cv2.imread(path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        corrected = correct_skew_combined(gray)

        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, corrected)

print("✅ اصلاح زاویه با ترکیب minAreaRect + Hough انجام شد.")


✅ اصلاح زاویه با ترکیب minAreaRect + Hough انجام شد.


### کات دادن یک هشتم اول پلاک

In [7]:
import os
from PIL import Image

# مسیرهای ورودی و خروجی
input_dir = 'output_phase2'
output_dir = 'output_phase2.1'

# ساخت پوشه خروجی اگر وجود نداشت
os.makedirs(output_dir, exist_ok=True)

# پردازش هر تصویر در پوشه ورودی
for filename in os.listdir(input_dir):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tif')):
        img_path = os.path.join(input_dir, filename)
        img = Image.open(img_path)

        # ابعاد تصویر
        width, height = img.size

        # عرض هر تکه
        slice_width = width // 8

        # برش تصویر: حذف تیکه اول (از x=0 تا x=slice_width)
        # نگه داشتن بخش باقی‌مانده (از x=slice_width تا x=width)
        cropped_img = img.crop((slice_width, 0, width, height))

        # ذخیره تصویر خروجی
        output_path = os.path.join(output_dir, filename)
        cropped_img.save(output_path)

print("پردازش تمام شد.")


پردازش تمام شد.


### باینری کردن عکس ها

In [1]:
import os
import cv2
import numpy as np

input_folder = 'output_phase2.1'
output_folder = 'output_phase2.2'
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
        img_path = os.path.join(input_folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        
        # 1. نویززدایی با فیلتر غیرمحلی (Non-local Means)
        denoised = cv2.fastNlMeansDenoising(img, h=10, templateWindowSize=7, searchWindowSize=21)
        
        # 2. افزایش کنتراست با CLAHE (بهتر از اکولایزیشن ساده)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        contrast_enhanced = clahe.apply(denoised)
        
        # 3. فیلتر گوسی برای نرم کردن (با هسته کوچکتر)
        blurred = cv2.GaussianBlur(contrast_enhanced, (3,3), 0)
        
        # 4. آستانه‌گذاری تطبیقی با پارامترهای بهینه‌تر
        binary_img = cv2.adaptiveThreshold(
            blurred,
            255,
            cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY_INV,
            11,  # اندازه بلوک - باید کوچکتر باشد برای حفظ جزئیات
            2    # مقدار ثابت - کاهش یافته برای جلوگیری از سیاه شدن نواحی روشن
        )
        
        # 5. مورفولوژی برای حذف نویزهای کوچک
        kernel = np.ones((2,2), np.uint8)
        processed = cv2.morphologyEx(binary_img, cv2.MORPH_OPEN, kernel)
        
        # معکوس کردن تصویر برای داشتن متن سفید روی پس زمینه سیاه
        final_img = cv2.bitwise_not(processed)
        
        # ذخیره خروجی
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, final_img)

print("پردازش پیشرفته و تبدیل به باینری با حفظ جزئیات انجام شد.")

پردازش پیشرفته و تبدیل به باینری با حفظ جزئیات انجام شد.


In [ ]:
import os
import cv2
import numpy as np

input_folder = 'output_phase2.1'
output_folder = 'output_phase2.2'
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
        img_path = os.path.join(input_folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        
        # 1. پیش‌پردازش اولیه با حفظ جزئیات
        denoised = cv2.fastNlMeansDenoising(img, h=5, templateWindowSize=7, searchWindowSize=21)
        
        # 2. افزایش کنتراست هوشمند
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(12,12))
        contrast_enhanced = clahe.apply(denoised)
        
        # 3. بهبود لبه‌ها با فیلتر ویژه
        blurred = cv2.bilateralFilter(contrast_enhanced, d=9, sigmaColor=75, sigmaSpace=75)
        
        # 4. روش ترکیبی آستانه‌گذاری
        # مرحله اول: Adaptive Threshold
        binary_adaptive = cv2.adaptiveThreshold(
            blurred,
            255,
            cv2.ADAPTIVE_THRESH_MEAN_C,
            cv2.THRESH_BINARY_INV,
            blockSize=9,
            C=2
        )
        
        # مرحله دوم: Otsu Threshold
        _, binary_otsu = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        
        # مرحله سوم: ترکیب با وزن‌دهی
        combined = cv2.addWeighted(binary_adaptive, 0.7, binary_otsu, 0.3, 0)
        
        # 5. بهبود نهایی با تکنیک Super-Resolution (اختیاری)
        # اگر opencv با contrib نصب باشد:
        # super_res = cv2.dnn_superres.DnnSuperResImpl_create()
        # super_res.readModel('EDSR_x4.pb')
        # super_res.setModel('edsr', 4)
        # final_img = super_res.upsample(combined)
        
        # 6. ذخیره نتیجه
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, cv2.bitwise_not(combined))

print("پردازش حرفه‌ای با حفظ حداکثر کیفیت انجام شد.")

پردازش حرفه‌ای با حفظ حداکثر کیفیت انجام شد.


### در اوردن کاراکتر های هر عکس

In [26]:
import os
import cv2
import numpy as np

input_folder = "output_phase2.2"
output_folder = "output_phase2.3"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for filename in os.listdir(input_folder):
    if not filename.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff")):
        continue

    path = os.path.join(input_folder, filename)
    img_original = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if img_original is None:
        print(f"⛔ نمی‌توان تصویر {filename} را خواند.")
        continue

    # بزرگنمایی ×3 برای حفظ جزئیات
    scale = 3
    img_highres = cv2.resize(img_original, (img_original.shape[1]*scale, img_original.shape[0]*scale), interpolation=cv2.INTER_CUBIC)

    # فیلتر گوسی برای کاهش نویز
    img_blur = cv2.GaussianBlur(img_highres, (3, 3), 0)

    # آستانه‌گذاری تطبیقی برای تطبیق با نورهای غیر یکنواخت
    img_bin = cv2.adaptiveThreshold(
        img_blur, 255,
        cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY_INV,
        15, 8
    )

    # حذف نویزهای ریز
    kernel = np.ones((2, 2), np.uint8)
    img_bin = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, kernel)

    # پیدا کردن تمام کانتورها (نه فقط خارجی)
    contours, _ = cv2.findContours(img_bin, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    bounding_boxes = [cv2.boundingRect(c) for c in contours]

    if len(bounding_boxes) == 0:
        continue

    heights = [h for (_, _, _, h) in bounding_boxes]
    widths = [w for (_, _, w, _) in bounding_boxes]
    mean_height = np.mean(heights)
    mean_width = np.mean(widths)

    filtered_boxes = []
    for x, y, w, h in bounding_boxes:
        aspect_ratio = h / float(w + 1e-5)
        if w < 6 or h < 15:
            continue
        if h < 0.4 * mean_height or h > 2.2 * mean_height:
            continue
        if w > 3.0 * mean_width:
            continue
        if not (0.5 < aspect_ratio < 7.0):
            continue

        # بررسی چگالی پیکسل سفید (برای حذف نویز)
        roi = img_bin[y:y + h, x:x + w]
        white_ratio = np.sum(roi == 255) / (roi.shape[0] * roi.shape[1])
        if white_ratio < 0.05 or white_ratio > 0.9:
            continue

        filtered_boxes.append((x, y, w, h))

    # مرتب‌سازی از چپ به راست
    filtered_boxes = sorted(filtered_boxes, key=lambda b: b[0])
    pad = 6
    count = 0

    for i, (x, y, w, h) in enumerate(filtered_boxes):
        x_new = max(x - pad, 0)
        y_new = max(y - pad, 0)
        w_new = min(w + 2 * pad, img_highres.shape[1] - x_new)
        h_new = min(h + 2 * pad, img_highres.shape[0] - y_new)

        char_img = img_highres[y_new:y_new + h_new, x_new:x_new + w_new]

        # resize و شارپ‌سازی
        char_img_resized = cv2.resize(char_img, (64, 128), interpolation=cv2.INTER_CUBIC)

        kernel_sharp = np.array([[0, -1, 0],
                                 [-1, 5, -1],
                                 [0, -1, 0]])
        char_img_resized = cv2.filter2D(char_img_resized, -1, kernel_sharp)

        out_name = f"{os.path.splitext(filename)[0]}_char{count + 1}.png"
        cv2.imwrite(os.path.join(output_folder, out_name), char_img_resized)
        count += 1

    print(f"✅ {filename}: {count} کاراکتر نهایی ذخیره شد.")

print("🎯 پردازش کامل شد - دقت بالا و تعداد بیشتر.")


✅ 1.png: 8 کاراکتر نهایی ذخیره شد.
✅ 10.png: 8 کاراکتر نهایی ذخیره شد.
✅ 100.png: 8 کاراکتر نهایی ذخیره شد.
✅ 101.png: 7 کاراکتر نهایی ذخیره شد.
✅ 102.png: 7 کاراکتر نهایی ذخیره شد.
✅ 103.png: 2 کاراکتر نهایی ذخیره شد.
✅ 104.png: 8 کاراکتر نهایی ذخیره شد.
✅ 105.png: 10 کاراکتر نهایی ذخیره شد.
✅ 106.png: 9 کاراکتر نهایی ذخیره شد.
✅ 107.png: 7 کاراکتر نهایی ذخیره شد.
✅ 108.png: 7 کاراکتر نهایی ذخیره شد.
✅ 109.png: 7 کاراکتر نهایی ذخیره شد.
✅ 11.png: 8 کاراکتر نهایی ذخیره شد.
✅ 110.png: 10 کاراکتر نهایی ذخیره شد.
✅ 111.png: 7 کاراکتر نهایی ذخیره شد.
✅ 112.png: 3 کاراکتر نهایی ذخیره شد.
✅ 113.png: 8 کاراکتر نهایی ذخیره شد.
✅ 114.png: 11 کاراکتر نهایی ذخیره شد.
✅ 115.png: 8 کاراکتر نهایی ذخیره شد.
✅ 116.png: 5 کاراکتر نهایی ذخیره شد.
✅ 117.png: 8 کاراکتر نهایی ذخیره شد.
✅ 118.png: 8 کاراکتر نهایی ذخیره شد.
✅ 119.png: 7 کاراکتر نهایی ذخیره شد.
✅ 12.png: 4 کاراکتر نهایی ذخیره شد.
✅ 120.png: 9 کاراکتر نهایی ذخیره شد.
✅ 121.png: 5 کاراکتر نهایی ذخیره شد.
✅ 122.png: 9 کاراکتر نهایی ذخیره شد.
✅ 1

### تنظیم سایز تصاویر

In [24]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# مسیرها
input_dir = 'output_phase2.3'
output_dir = 'output_phase2.4'
os.makedirs(output_dir, exist_ok=True)

def clean_and_center_image(img, size=(28, 28)):
    # تبدیل به خاکستری اگر رنگی بود
    if len(img.shape) == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # باینری کردن (متن سیاه، زمینه سفید)
    _, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # حذف نویز با Morphology
    kernel = np.ones((2, 2), np.uint8)
    clean = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)

    # پیدا کردن بزرگ‌ترین کانتور (کاراکتر اصلی)
    contours, _ = cv2.findContours(clean, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        return np.ones(size, dtype=np.uint8) * 255  # برگرد تصویر سفید خالی

    # انتخاب بزرگ‌ترین کانتور
    main_contour = max(contours, key=cv2.contourArea)
    mask = np.zeros_like(clean)
    cv2.drawContours(mask, [main_contour], -1, 255, thickness=cv2.FILLED)

    # فقط کاراکتر اصلی نگه داشته بشه
    isolated = cv2.bitwise_and(clean, mask)

    # کراپ به اطراف کاراکتر
    x, y, w, h = cv2.boundingRect(main_contour)
    char_img = isolated[y:y+h, x:x+w]

    # ریسایز با حفظ نسبت
    h_new, w_new = char_img.shape
    scale = min((size[0] - 4) / h_new, (size[1] - 4) / w_new)
    char_resized = cv2.resize(char_img, (int(w_new * scale), int(h_new * scale)), interpolation=cv2.INTER_AREA)

    # قرار دادن وسط تصویر سفید
    final_img = np.ones(size, dtype=np.uint8) * 255
    h_final, w_final = char_resized.shape
    y_offset = (size[0] - h_final) // 2
    x_offset = (size[1] - w_final) // 2
    final_img[y_offset:y_offset+h_final, x_offset:x_offset+w_final] = 255 - char_resized  # برعکسش کنیم چون اینورت شده بود

    return final_img

# پردازش تصاویر
for filename in tqdm(os.listdir(input_dir)):
    input_path = os.path.join(input_dir, filename)
    output_path = os.path.join(output_dir, filename)

    img = cv2.imread(input_path)
    if img is None:
        continue

    cleaned_img = clean_and_center_image(img)
    cv2.imwrite(output_path, cleaned_img)


100%|██████████| 1464/1464 [00:00<00:00, 1732.57it/s]


# فاز 3

### آماده سازی داده ها

In [5]:
import os
from PIL import Image

In [6]:


input_dir = 'alpha'
output_dir = 'output_phase3'

os.makedirs(output_dir, exist_ok=True)

for folder_name in os.listdir(input_dir):
    input_folder_path = os.path.join(input_dir, folder_name)
    
    if os.path.isdir(input_folder_path):
        output_folder_path = os.path.join(output_dir, folder_name)
        os.makedirs(output_folder_path, exist_ok=True)

        for img_name in os.listdir(input_folder_path):
            input_img_path = os.path.join(input_folder_path, img_name)
            output_img_path = os.path.join(output_folder_path, img_name)

            try:
                with Image.open(input_img_path) as img:
                    resized_img = img.resize((28, 28), Image.LANCZOS)  # ← الگوریتم بهتر
                    resized_img.save(output_img_path)
            except Exception as e:
                print(f"خطا در پردازش عکس {input_img_path}: {e}")


### اموزش مدل

In [7]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from skimage.feature import hog
import joblib
from tqdm import tqdm

In [8]:


input_dir = 'output_phase3'
output_dir = 'output_phase3.1'
os.makedirs(output_dir, exist_ok=True)

image_size = (28, 28)
orientations = 9
pixels_per_cell = (8, 8)
cells_per_block = (2, 2)

X = []
y = []

label_names = sorted(os.listdir(input_dir))
label_map = {name: idx for idx, name in enumerate(label_names)}

def preprocess_image(img):
    # تبدیل به سیاه و سفید
    _, binary = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # حذف فضای اضافی
    x, y_, w, h = cv2.boundingRect(binary)
    cropped = binary[y_:y_+h, x:x+w]

    # تغییر اندازه
    resized = cv2.resize(cropped, image_size)
    return resized

def augment_image(img):
    rows, cols = img.shape
    angle = np.random.uniform(-10, 10)
    M = cv2.getRotationMatrix2D((cols/2, rows/2), angle, 1.0)
    return cv2.warpAffine(img, M, (cols, rows), borderValue=255)

print("در حال پردازش تصاویر...")

for label_name in tqdm(label_names):
    folder_path = os.path.join(input_dir, label_name)
    if not os.path.isdir(folder_path):
        continue
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        try:
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue

            # پردازش اصلی
            pre_img = preprocess_image(img)

            # استخراج ویژگی اصلی
            features = hog(pre_img,
                           orientations=orientations,
                           pixels_per_cell=pixels_per_cell,
                           cells_per_block=cells_per_block,
                           block_norm='L2-Hys',
                           visualize=False)
            X.append(features)
            y.append(label_map[label_name])

            # Augmented نسخه چرخش‌دار
            aug_img = augment_image(pre_img)
            features_aug = hog(aug_img,
                               orientations=orientations,
                               pixels_per_cell=pixels_per_cell,
                               cells_per_block=cells_per_block,
                               block_norm='L2-Hys',
                               visualize=False)
            X.append(features_aug)
            y.append(label_map[label_name])

        except Exception as e:
            print(f"⚠️ خطا در پردازش {img_path}: {e}")

X = np.array(X)
y = np.array(y)

print(f"\n📊 مجموع نمونه‌ها (شامل نسخه‌های چرخش‌دار): {X.shape[0]}")

# کاهش ابعاد با PCA
print("⚙️ در حال اعمال PCA برای کاهش بعد ویژگی‌ها...")
pca = PCA(n_components=50)
X = pca.fit_transform(X)

# تقسیم آموزش/تست
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# آموزش SVM
print("\n🎓 آموزش SVM...")
svm_model = SVC(kernel='linear', probability=True)
svm_model.fit(X_train, y_train)
svm_acc = accuracy_score(y_test, svm_model.predict(X_test))
print(f"🎯 دقت SVM: {svm_acc:.2f}")

# آموزش Decision Tree
print("\n🎓 آموزش درخت تصمیم...")
dt_model = DecisionTreeClassifier(max_depth=15)
dt_model.fit(X_train, y_train)
dt_acc = accuracy_score(y_test, dt_model.predict(X_test))
print(f"🌳 دقت درخت تصمیم: {dt_acc:.2f}")

# ذخیره مدل‌ها و PCA
joblib.dump(svm_model, os.path.join(output_dir, 'svm_model.joblib'))
joblib.dump(dt_model, os.path.join(output_dir, 'decision_tree_model.joblib'))
joblib.dump(pca, os.path.join(output_dir, 'pca_transform.joblib'))

print(f"\n✅ مدل‌ها و PCA در پوشه {output_dir} ذخیره شدند.")


در حال پردازش تصاویر...


100%|██████████| 43/43 [00:01<00:00, 26.81it/s]



📊 مجموع نمونه‌ها (شامل نسخه‌های چرخش‌دار): 8598
⚙️ در حال اعمال PCA برای کاهش بعد ویژگی‌ها...

🎓 آموزش SVM...
🎯 دقت SVM: 0.87

🎓 آموزش درخت تصمیم...
🌳 دقت درخت تصمیم: 0.64

✅ مدل‌ها و PCA در پوشه output_phase3.1 ذخیره شدند.


# فاز 4

In [12]:
import os
import cv2
import numpy as np
from joblib import load

# مسیرها
input_folder = 'output_phase2.4'
model_folder = 'output_phase3.2'
output_folder = 'output_phase4'

# ایجاد پوشه خروجی
os.makedirs(output_folder, exist_ok=True)

# بارگذاری مدل‌ها
svm_model = load(os.path.join(model_folder, 'svm_model.joblib'))
tree_model = load(os.path.join(model_folder, 'tree_model.joblib'))

# مشخص کردن ابعاد مورد انتظار مدل
expected_features = svm_model.n_features_in_
img_width = 42
img_height = expected_features // img_width  # مثلاً 2352 // 42 = 56

# پردازش تصاویر و پیش‌بینی
results = []

for filename in sorted(os.listdir(input_folder)):
    if not filename.lower().endswith('.png'):
        continue

    filepath = os.path.join(input_folder, filename)
    img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)

    if img is None:
        print(f"نمی‌توان تصویر را بخوانم: {filepath}")
        continue

    # تغییر اندازه به اندازه مورد انتظار مدل
    resized = cv2.resize(img, (img_width, img_height), interpolation=cv2.INTER_AREA)

    # تبدیل به بردار ویژگی
    features = resized.flatten().reshape(1, -1)

    # پیش‌بینی با دو مدل
    svm_pred = svm_model.predict(features)[0]
    tree_pred = tree_model.predict(features)[0]

    # ذخیره نتیجه
    results.append((filename, svm_pred, tree_pred))

# ذخیره در فایل
output_file = os.path.join(output_folder, 'results.txt')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write('filename\tSVM_prediction\tDecisionTree_prediction\n')
    for filename, svm_pred, tree_pred in results:
        f.write(f"{filename}\t{svm_pred}\t{tree_pred}\n")

print(f"✅ نتایج در {output_file} ذخیره شد.")


✅ نتایج در output_phase4\results.txt ذخیره شد.


# فاز 5

In [15]:
import os
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# مسیر فایل نتایج فاز ۴
input_txt_path = "output_phase4/results.txt"

# بررسی وجود فایل
if not os.path.exists(input_txt_path):
    raise FileNotFoundError(f"فایل {input_txt_path} پیدا نشد.")

# خواندن فایل متنی و ساخت دیتافریم
with open(input_txt_path, "r", encoding="utf-8") as f:
    lines = f.readlines()

data = []
for line in lines[1:]:  # رد کردن هدر
    parts = line.strip().split("\t")
    if len(parts) == 3:
        data.append(parts)

df = pd.DataFrame(data, columns=["name", "SVM_prediction", "DecisionTree_prediction"])

# استخراج برچسب واقعی از نام فایل (مثلاً 104_char2.png → 104)
df["true_label"] = df["name"].apply(lambda x: x.split("_")[0])

# آماده‌سازی برای ارزیابی
y_true = df["true_label"]
svm_pred = df["SVM_prediction"]
tree_pred = df["DecisionTree_prediction"]

# محاسبه معیارها
svm_acc = accuracy_score(y_true, svm_pred)
tree_acc = accuracy_score(y_true, tree_pred)
svm_report = classification_report(y_true, svm_pred, zero_division=0)
tree_report = classification_report(y_true, tree_pred, zero_division=0)
svm_cm = confusion_matrix(y_true, svm_pred)
tree_cm = confusion_matrix(y_true, tree_pred)

# ساخت پوشه خروجی
output_dir = "output_phase5"
os.makedirs(output_dir, exist_ok=True)

# ذخیره خروجی‌ها در فایل متنی
with open(os.path.join(output_dir, "evaluation.txt"), "w", encoding="utf-8") as f:
    f.write(f"SVM Accuracy: {svm_acc:.4f}\n")
    f.write(f"Decision Tree Accuracy: {tree_acc:.4f}\n\n")

    f.write("=== SVM Classification Report ===\n")
    f.write(svm_report + "\n")
    f.write("=== SVM Confusion Matrix ===\n")
    f.write(str(svm_cm) + "\n\n")

    f.write("=== Decision Tree Classification Report ===\n")
    f.write(tree_report + "\n")
    f.write("=== Decision Tree Confusion Matrix ===\n")
    f.write(str(tree_cm) + "\n")

# --- چاپ در ترمینال ---
print("✅ ارزیابی مدل‌ها تکمیل شد.")
print(f"SVM Accuracy: {svm_acc:.4f}")
print(f"Decision Tree Accuracy: {tree_acc:.4f}")


✅ ارزیابی مدل‌ها تکمیل شد.
SVM Accuracy: 0.0000
Decision Tree Accuracy: 0.0000
